In [22]:
import pyarrow as pa
import pyarrow.parquet as pq
import geopandas as gpd
from pyogrio import list_layers
import pandas as pd

In [3]:
onnet_lakes = 'O:/PRIV/CPHEA/PESD/COR/CORFILES/Geospatial_Library_Projects/LakeCat/OnNetLakeWatersheds/OnNetWorkWatersheds.gpkg'
list_layers(onnet_lakes)

array([['OnNetWatersheds1', 'Unknown'],
       ['OnNetWatersheds2', 'Polygon'],
       ['OnNetWatersheds3', 'Unknown'],
       ['OnNetWatersheds4', 'Unknown'],
       ['OnNetWatersheds5', 'Unknown'],
       ['OnNetWatersheds6', 'Unknown'],
       ['OnNetWatersheds7', 'Unknown'],
       ['OnNetWatersheds8', 'Polygon'],
       ['OnNetWatersheds9', 'Unknown'],
       ['OnNetWatersheds10', 'Polygon'],
       ['OnNetWatersheds11', 'Unknown'],
       ['OnNetWatersheds12', 'Unknown'],
       ['OnNetWatersheds13', 'Unknown'],
       ['OnNetWatersheds14', 'Unknown'],
       ['OnNetWatersheds15', 'Unknown'],
       ['OnNetWatersheds16', 'Unknown'],
       ['OnNetWatersheds17', 'Unknown'],
       ['OnNetWatersheds18', 'Unknown'],
       ['OnNetWatersheds19', 'Unknown'],
       ['OnNetWatersheds20', 'Unknown'],
       ['OnNetWatersheds21', 'Unknown']], dtype=object)

In [7]:
# lakes = gpd.read_file('E:/GitProjects/NARS/NARS.gdb', 'OnNetWatersheds1')
lakes = gpd.read_file('E:/WorkingData/main_OnNetWatersheds1.shp')

In [74]:
temp = gpd.read_file('E:/WorkingData/lakes27.shp')

C:\Users\mweber\AppData\Local\miniforge3\envs\HyRiver\Lib\site-packages\pyogrio\raw.py:194: RuntimeWarning: E:/WorkingData/lakes27.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  result = ogr_read(


In [76]:
temp = gpd.read_file('O:/PRIV/CPHEA/PESD/COR/CORFILES/Geospatial_Library_Projects/LakeCat/Watersheds_Framework/allBasins.shp')

In [81]:
list(temp)

['COMID', 'geometry']

In [80]:
temp = temp[['COMID','geometry']]

In [78]:
list(lakes)

['COMID', 'WBCOMID', 'geometry']

In [82]:
lakes = lakes[['WBCOMID','geometry']]
lakes = lakes.rename(columns={'WBCOMID': 'COMID'})
list(lakes)

['COMID', 'geometry']

In [83]:
lakes = pd.concat([lakes, temp])

In [ ]:
wb = gpd.read_file('G:/NHDPlusV21/NHDPlusNationalData/NHDPlusV21_National_Seamless_Flattened_Lower48.gdb',
                   layer='NHDWaterbody')

In [86]:
backup = lakes

In [87]:
list(wb)

['COMID',
 'FDATE',
 'RESOLUTION',
 'GNIS_ID',
 'GNIS_NAME',
 'AREASQKM',
 'ELEVATION',
 'REACHCODE',
 'FTYPE',
 'FCODE',
 'Shape_Length',
 'Shape_Area',
 'ONOFFNET',
 'PurpCode',
 'PurpDesc',
 'MeanDepth',
 'LakeVolume',
 'MaxDepth',
 'MeanDUsed',
 'MeanDCode',
 'LakeArea',
 'geometry']

In [88]:
lakes = pd.merge(lakes, wb[['COMID','GNIS_ID','GNIS_NAME','REACHCODE']], on='COMID',how="left")

In [89]:
lakes.head()

,COMID,geometry,GNIS_ID,GNIS_NAME,REACHCODE
0,1735036,"POLYGON ((2129595 2748554.99, 2129594.999 2748...",,,01020005001969
1,1734978,"POLYGON ((2094104.999 2748374.99, 2094104.998 ...",,,01020005001966
2,6159761,"POLYGON ((1934415 2354204.99, 1934414.999 2354...",209227,New City Pond,01100002000642
3,6160289,"POLYGON ((1928055 2353964.99, 1928054.999 2353...",211151,State Line Pond,01100002000646
4,1735034,"POLYGON ((2129384.99 2748824.99, 2129355 27488...",,,01020005001970


In [90]:
lakes.to_parquet('D:/LakeCatWatersheds.parquet')

In [96]:
lakes.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 382245 entries, 0 to 382244
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype   
---  ------     --------------   -----   
 0   COMID      382245 non-null  int64   
 1   geometry   382245 non-null  geometry
 2   GNIS_ID    382245 non-null  object  
 3   GNIS_NAME  382245 non-null  object  
 4   REACHCODE  382245 non-null  object  
dtypes: geometry(1), int64(1), object(3)
memory usage: 14.6+ MB


In [91]:
import pyarrow.parquet as pq
from geopandas.io.arrow import _arrow_to_geopandas

In [98]:
parquet_file = pq.ParquetFile('D:/LakeCatWatersheds.parquet')

In [99]:
parquet_file.schema_arrow.names

['COMID', 'geometry', 'GNIS_ID', 'GNIS_NAME', 'REACHCODE']

In [100]:
parquet_file.num_row_groups

1

In [103]:
wb = pq.read_table('D:/LakeCatWatersheds.parquet', filters=[('COMID','==',1735034)])

In [105]:
geopandas_gdf = _arrow_to_geopandas(wb)

In [106]:
geopandas_gdf.head()

,COMID,geometry,GNIS_ID,GNIS_NAME,REACHCODE
0,1735034,"POLYGON ((2129384.99 2748824.99, 2129355 27488...",,,01020005001970
